In [3]:
import os, re, copy
from pathlib import Path
import pandas as pd
import requests 
from stop_words import get_stop_words
import warnings 
from pathlib import Path
from bs4 import BeautifulSoup 
import json
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

In [31]:
url= f'https://scholar.google.com/scholar?cites=14392702990254182637&as_sdt=5,47&sciodt=0,47&hl=en' # user manual input
citation_key =f'de1985does' # user manual input

headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
page_contents=requests.get(url,headers=headers)
soup=BeautifulSoup(page_contents.text,'html.parser')

citation_dict={}
def cited_by_gscholar(dsoup):
    cited_by_link_list=[]
    for item in dsoup.find_all('div',{"class" : "gs_ri"}):
        cited_by_link_dict={}
        authors=[]
        for el in item.extract().find_all('a'):
            if 'https' in el.get('href').split(':'):
                if len(set(['View as HTML', 'Cached']) & set([str(el.text)])) ==0:
                    cited_by_link_dict['article_title']=el.text
                    cited_by_link_dict['article_link']=el.get('href')
            if '/citations?user' in el.get('href'):
                authors.append(el.text)
            elhref = el.get('href')
            if '/scholar?cites' in elhref.split('='):
                cited_by_link_dict['cited_by']=elhref
            if 'by' in el.text.split():
                #print(el.text.split())
                citation_cnt=re.findall(r'\d+', el.text)[0]
                cited_by_link_dict['citations_count']=citation_cnt
                cited_by_link_list.append(cited_by_link_dict)
        cited_by_link_dict['auhtors']=authors
    return cited_by_link_list

dsoup = copy.deepcopy(soup)
dsoup= cited_by_gscholar(dsoup)
citation_dict['citation_key']=citation_key
citation_dict['google_scholar_link']=url
citation_dict['cited_by']=dsoup
df = pd.DataFrame.from_dict(citation_dict)
df.to_csv('academic_citation_network.csv', mode='a')
citation_dict

{'citation_key': 'de1985does',
 'google_scholar_link': 'https://scholar.google.com/scholar?cites=14392702990254182637&as_sdt=5,47&sciodt=0,47&hl=en',
 'cited_by': [{'article_title': 'Machine learning techniques and data for stock market forecasting: A literature review',
   'article_link': 'https://www.sciencedirect.com/science/article/pii/S0957417422001452',
   'cited_by': '/scholar?cites=14110825883629850631&as_sdt=5,47&sciodt=0,47&hl=en',
   'citations_count': '299',
   'auhtors': ['MM Kumbure', 'C Lohrmann', 'P Luukka']},
  {'article_title': 'The noise trader approach to finance',
   'article_link': 'https://www.aeaweb.org/articles?id=10.1257/jep.4.2.19',
   'cited_by': '/scholar?cites=15147237888813115420&as_sdt=5,47&sciodt=0,47&hl=en',
   'citations_count': '2669',
   'auhtors': ['A Shleifer']},
  {'article_title': 'Risks and returns of cryptocurrency',
   'article_link': 'https://academic.oup.com/rfs/article-abstract/34/6/2689/5912024',
   'cited_by': '/scholar?cites=15325183468